## Nesesary installs 

In [1]:
#!pip install pandas sqlalchemy
#!pip install openpyxl

## Task description
Partners receive bonuses for every client that they introduce to the company. 
Assume that the company has 3 active partners.
Partner paid on a monthly based - for every introduced client partner gets 0.5% of the amount deposited by the client excluding withdrawals. But, with the condition that if the client withdraws the amount more than 3 months after the amount was deposited – such withdrawals are not included in the partner’s bonus reduction. 
For example, in August partner’s client deposited $1000 and withdraw $100. Partner gets (1000-100)*0.5%=$4.5. But in case the $100 withdrawal is from the deposit made by the client in April, in this case, the partner’s bonus is (1000)*0.5%=$5

Task: write an SQL script that calculates the monthly bonus amount for each partner.  

## Baseline solution 

In [2]:
# Example of sql code to create table if not created
'''
CREATE TABLE transactions (
    transaction_id INT PRIMARY KEY,
    partner_id INT REFERENCES partners(partner_id),
    client_id INT,
    deposit_amount DECIMAL(10,2),
    withdrawal_amount DECIMAL(10,2),
    transaction_date DATE
);
''';

In [3]:
# Basline sql-code solution 
sql_solution = \
'''
WITH EffectiveDeposits AS (
    SELECT 
        t1.manager_id AS partner,
        t1.client_id,
        strftime('%Y-%m', t1.date) AS year_month,
        t1.currency,
        t1.date AS deposit_date,
        t1.amount AS deposit_amount,
        COALESCE(SUM(CASE WHEN julianday(t2.date) - julianday(t1.date) <= 90 THEN t2.amount ELSE 0 END), 0) AS valid_withdrawals
    FROM 
        transactions t1
    LEFT JOIN 
        transactions t2 ON t1.client_id = t2.client_id AND t2.amount < 0 AND t2.date > t1.date AND t1.currency = t2.currency
    WHERE 
        t1.amount > 0
    GROUP BY 
        t1.manager_id, t1.client_id, t1.date, t1.amount, t1.currency
)

SELECT 
    partner,
    year_month,
    currency,
    SUM(deposit_amount + valid_withdrawals) * 0.005 AS bonus
FROM 
    EffectiveDeposits
GROUP BY 
    partner, year_month, currency;
'''

### Python imports 

In [4]:
import pandas as pd
from sqlalchemy import create_engine

In [5]:
# Load the spreadsheet into a DataFrame
df = pd.read_excel("Test task.xlsx",sheet_name ='data')

#### DB creation 

In [6]:
# Create an SQLite database in memory (you can replace this with any other database URL)
engine = create_engine('sqlite:///:memory:')

In [7]:
# Write the DataFrame to the database as a table (change table name if needed)
df.to_sql('transactions', engine, index=False, if_exists='replace')

142

#### SQL-execution

In [8]:
# Check that everything works fine 
df_result = pd.read_sql("SELECT * FROM transactions LIMIT 5", engine)
df_result

,manager_id,id,client_id,amount,currency,type,date
0,IB1,259446301,ID1,4999.0,USD,deposit,2020-02-19 00:00:00.000000
1,IB2,268645095,ID2,300000.0,USD,deposit,2021-10-04 00:00:00.000000
2,IB2,271029259,ID3,1800000.0,USD,deposit,2021-11-22 00:00:00.000000
3,IB1,266942215,ID1,1000.0,USD,deposit,2021-07-19 00:00:00.000000
4,IB1,271592654,ID1,2100.0,USD,withdrawal,2021-12-02 00:00:00.000000


In [9]:
# Answer on besline solution 
df_result = pd.read_sql(sql_solution, engine)
df_result

,partner,year_month,currency,bonus
0,IB1,2020-01,USD,10.70500
1,IB1,2020-02,EUR,0.50000
2,IB1,2020-02,USD,24.99500
3,IB1,2020-03,USD,149.97000
4,IB1,2020-04,EUR,2.25000
5,IB1,2020-04,USD,341.00000
6,IB1,2020-05,USD,310.00000
7,IB1,2020-06,USD,199.00000
8,IB1,2020-07,EUR,1.25000
9,IB1,2020-11,EUR,2.50000


In [10]:
# Filter on one month where bouth partners occure
df_result[df_result['year_month'] == '2021-11']

,partner,year_month,currency,bonus
23,IB1,2021-11,USD,188.0
35,IB2,2021-11,USD,20500.0
47,IB3,2021-11,EUR,90.0
48,IB3,2021-11,USD,330.0


## Sophisticated solution

In [11]:
# SQL solution with conversion bonus to EUR
sql_solution = \
'''
WITH EffectiveDeposits AS (
    SELECT 
        t1.manager_id AS partner,
        t1.client_id,
        strftime('%Y-%m', t1.date) AS year_month,
        t1.date AS deposit_date,
        t1.amount * er1.rate_to_eur AS deposit_amount_in_eur,
        COALESCE(SUM(CASE WHEN julianday(t2.date) - julianday(t1.date) <= 90 THEN t2.amount * er2.rate_to_eur ELSE 0 END), 0) AS valid_withdrawals_in_eur
    FROM 
        transactions t1
    LEFT JOIN 
        transactions t2 ON t1.client_id = t2.client_id AND t2.amount < 0 AND t2.date > t1.date
    JOIN 
        exchange_rates er1 ON t1.currency = er1.currency
    LEFT JOIN 
        exchange_rates er2 ON t2.currency = er2.currency
    WHERE 
        t1.amount > 0
    GROUP BY 
        t1.manager_id, t1.client_id, t1.date, t1.amount
)

SELECT 
    partner,
    year_month,
    SUM(deposit_amount_in_eur + valid_withdrawals_in_eur) * 0.005 AS bonus_in_eur
FROM 
    EffectiveDeposits
GROUP BY 
    partner, year_month;
'''

#### DB creation 

In [12]:
# Example of sql code to create table for exchange rates
'''
CREATE TABLE exchange_rates (
    currency TEXT PRIMARY KEY,
    rate_to_eur REAL
);

INSERT INTO exchange_rates (currency, rate_to_eur)
VALUES 
    ('USD', 0.85),
    ('GBP', 1.12),  
    ('EUR', 1),
    ('ZAR', 0.06);    
''';

In [13]:
# Create a new table for exchange rates
data = {
    'currency': ['USD', 'GBP', 'EUR', 'ZAR'],
    'rate_to_eur': [0.85, 1.17, 1, 0.06] 
}
exchange_rates_df = pd.DataFrame(data)

# Write the exchange_rates to the database as a new table
exchange_rates_df.to_sql('exchange_rates', engine, index=False, if_exists='replace')

# Query the database using the engine:
result = engine.execute("SELECT * FROM exchange_rates")
for row in result:
    print(row)

('USD', 0.85)
('GBP', 1.17)
('EUR', 1.0)
('ZAR', 0.06)


#### SQL-execution

In [14]:
# Get results 
df_result = pd.read_sql(sql_solution, engine)
df_result

,partner,year_month,bonus_in_eur
0,IB1,2020-01,9.099250
1,IB1,2020-02,21.745750
2,IB1,2020-03,127.474500
3,IB1,2020-04,292.100000
4,IB1,2020-05,263.500000
5,IB1,2020-06,169.150000
6,IB1,2020-07,1.250000
7,IB1,2020-11,23.750000
8,IB1,2021-01,27.156862
9,IB1,2021-02,70.941722


In [15]:
# Filter on one month where bouth partners occure
df_result[df_result['year_month'] == '2021-11']

,partner,year_month,bonus_in_eur
17,IB1,2021-11,159.8
24,IB2,2021-11,17425.0
34,IB3,2021-11,370.5
